In [ ]:
import os.path
import bs4
import urllib
import datetime
import json

In [ ]:
base = 'https://chicago.councilmatic.org/'
suffix = '/search/?selected_facets=topics_exact%3AZoning+Reclassification'
page_query = '&page={num}'
resp = urllib.request.urlopen(base + suffix + page_query.format(num=1))
soup = bs4.BeautifulSoup(resp.read(), "html5lib")

In [ ]:
def get_ords(link):
    ord_info = {}
    ord_info['ord'] = link.contents[0].split()[1]
    ord_info['href'] = link.get('href')
    ord_info['status'] = str(link.next_element.next_element.next_element.string).strip()
    
    cal = link.parent.next_sibling.next_sibling.next_sibling.next_sibling
    date_string = cal.find('i', class_='fa fa-fw fa-calendar-o').next_element.split('-')[0].strip()
    date = [int(x) for x in date_string.split('/')]
    ord_info['date'] = datetime.date(date[2], date[0], date[1])
    ord_info['year'] = ord_info['date'].year
    return ord_info


def get_locs(inner_soup, ord_info):
    '''get locations mentioned in ordinance'''
    locs = []
    for inner_link in inner_soup.find_all('ul', class_='list-unstyled'):
        if str(inner_link.previous_element.previous_element).strip() == 'Locations mentioned':
            for list_item in inner_link.find_all('li'):
                locs.append(list_item.string.strip())
    ord_info['locations'] = locs
    return ord_info['locations']


def get_sponsors(inner_soup):
    sponsors = []
    wards = []
    inner_link = inner_soup.find('div', class_='table-responsive')
    spo = 1
    war = 1
    for col in inner_link.find_all('td'):
        if col.find('a'):
            sponsors.append(col.a.string.strip())
            spo += 1
        if col.string:
            wards.append(col.string.strip())
            war +=1
    return sponsors,wards

In [ ]:
ord_list = []
page_num = 0
while True:
    page_num = page_num+1
    try:
        resp = urllib.request.urlopen(base + suffix + page_query.format(num=page_num))
        soup = bs4.BeautifulSoup(resp.read(), "html5lib")
        
        '''loop through each ordinance on main search page'''
        for link in soup.find_all('a', class_='small'):
            ord_info = get_ords(link)
            if ord_info['year'] < 2015:
                break
    
            '''go to ordinance page to get additional info'''
            inner_resp = urllib.request.urlopen(base+ord_info['href'])
            inner_soup = bs4.BeautifulSoup(inner_resp.read(), "html5lib")
    
            ord_info['locations'] = get_locs(inner_soup, ord_info)
            sponsors, spo_wards = get_sponsors(inner_soup)
            ord_info['sponsors'] = sponsors
            ord_info['sponsor_ward'] = spo_wards

            '''append dict to list'''

            ord_list.append(ord_info)
            
    except:
        break


In [ ]:
with open('ord_dicts.txt', 'w') as fout:
    fout.write(json.dumps(ord_list, indent=4, sort_keys=True, default=str))